### Solovay-Kitaev Decomposition

In this file, we create our own SK-decomposition code, and we run uL on it to see how many gates it takes.

In [87]:
# Global helpers

import numpy as np
from itertools import product

# Pauli matrices
X = np.array([[0,1],[1,0]], dtype=complex)
Y = np.array([[0,-1j],[1j,0]], dtype=complex)
Z = np.array([[1,0],[0,-1]], dtype=complex)

# 1) SU(2) projector
def to_su2(M):
    d = M.shape[0]
    sign, logdet = np.linalg.slogdet(M)
    phi = np.angle(sign * np.exp(logdet))
    return M * np.exp(-1j * phi / d)

# 2) random SU(2)
def random_su2():
    A = (np.random.randn(2,2) + 1j*np.random.randn(2,2)) / np.sqrt(2)
    Q, R = np.linalg.qr(A)
    D = np.diag(np.exp(-1j * np.angle(np.diag(R))))
    return to_su2(Q @ D)

# Basic gates and native set
def native_gates():
    H = 1/np.sqrt(2) * np.array([[1,1],[1,-1]], dtype=complex)
    T = np.array([[1,0],[0,np.exp(1j*np.pi/4)]], dtype=complex)
    return {'H': H, 'T': T, 'Tdg': T.conj().T}

gate_dict = native_gates()

# 3) Closed-form SolveForPhi
def SolveForPhi(theta, tol=1e-12):
    if abs(theta) < tol:
        return 0.0
    return 2 * np.arcsin(np.sqrt(np.sin(theta/4)))

# 4) Axis–angle extraction
def AxisAngle(U, tol=1e-8):
    cos_half = np.real(np.trace(U) / 2)
    theta = 2 * np.arccos(np.clip(cos_half, -1, 1))
    if abs(theta) < tol:
        return 0.0, np.array([0.0, 0.0, 1.0])
    M = (U - U.conj().T) / (2j * np.sin(theta/2))
    px = np.real((M[1,0] + M[0,1]) / 2)
    py = np.imag((M[1,0] - M[0,1]) / 2)
    pz = np.real((M[0,0] - M[1,1]) / 2)
    p = np.array([px, py, pz])
    return theta, p / np.linalg.norm(p)

# 5) Arbitrary-axis rotation
def AxisAngleRotation(n, phi):
    nx, ny, nz = np.array(n) / np.linalg.norm(n)
    I = np.eye(2, dtype=complex)
    return np.cos(phi/2) * I - 1j * np.sin(phi/2) * (nx*X + ny*Y + nz*Z)

# 6) Align rotation of Bloch vectors
def AlignRotation(u, v, tol=1e-8):
    u = np.array(u) / np.linalg.norm(u)
    v = np.array(v) / np.linalg.norm(v)
    if np.linalg.norm(u - v) < tol:
        return np.eye(2, dtype=complex)
    if np.linalg.norm(u + v) < tol:
        perp = np.cross(u, [1,0,0])
        if np.linalg.norm(perp) < tol:
            perp = np.cross(u, [0,1,0])
        return AxisAngleRotation(perp, np.pi)
    cosA = np.dot(u, v)
    A = np.arccos(np.clip(cosA, -1, 1))
    axis = np.cross(u, v)
    axis /= np.linalg.norm(axis)
    return AxisAngleRotation(axis, A)

# Elementary rotations

def Rz(theta): return np.diag([np.exp(-0.5j*theta), np.exp(0.5j*theta)])

def Rx(theta):
    c, s = np.cos(theta/2), np.sin(theta/2)
    return np.array([[c, -1j*s], [-1j*s, c]], dtype=complex)


def Ry(theta):
    c, s = np.cos(theta/2), np.sin(theta/2)
    return np.array([[c, -s], [s, c]], dtype=complex)



## Single qubit SK-algorithm

### Psuedo code for Solovay-Kitaev algorithm for single qubit gates


function Solovay-Kitaev(Gate = U, depth = n)

    if n == 0:

        return Basic_Approximation(U)

    else:

        Set $U_{n-1}$ = Solovay-Kitaev(U, n-1)

        Set $V, W$ = GC-Decompose($U U^\dagger_{n-1}$)

        Set $V_{n-1}$ = Solovay-Kitaev(V, n-1)

        Set $W_{n-1}$ = Solovay-Kitaev(W, n-1)

        return $U_n = V_{n-1} W_{n-1} V^\dagger_{n-1} W^\dagger_{n-1} U_{n-1}$



In [88]:
# Generate all sequences of length L

def generate_sequences(G, L):
    return [list(p) for p in product(G, repeat=L)]

# Compose and distance
def compose(seq, gate_dict):
    M = np.eye(2, dtype=complex)
    for g in seq:
        M = gate_dict[g] @ M
    return M

def distance(U, V):
    return np.linalg.svd(U - V, compute_uv=False)[0]

# 7) Precompute sequences projected into SU(2)
def precompute_sequences_and_unitaries(G, L0, gate_dict):
    precom = []
    for L in range(1, L0+1):
        for seq in generate_sequences(G, L):
            V = compose(seq, gate_dict)
            precom.append((seq, to_su2(V)))
    return precom

# 8) Simplify sequence (cancel inverses & collapse Ts)
def simplify_sequence(seq):
    stack = []
    for g in seq:
        if stack and ((stack[-1]=='H' and g=='H') or
                      (stack[-1]=='T' and g=='Tdg') or
                      (stack[-1]=='Tdg' and g=='T')):
            stack.pop()
        else:
            stack.append(g)
    out = []
    tc = 0
    for g in stack + ['_END_']:
        if g == 'T':
            tc += 1
        else:
            r = tc % 8
            out += ['T'] * r
            tc = 0
            if g in ['H', 'Tdg']:
                out.append(g)
    return out

# 9) Optional meet-in-the-middle fallback
def meet_in_middle(U, L0, gate_dict):
    L1 = L0 // 2
    L2 = L0 - L1
    part1 = [(seq, compose(seq, gate_dict)) for seq in generate_sequences(gate_dict.keys(), L1)]
    table = {}
    for seq, V in part1:
        key = tuple(np.round(V,6).flatten())
        table[key] = seq
    for seq2 in generate_sequences(gate_dict.keys(), L2):
        W = compose(seq2, gate_dict)
        target = to_su2(U @ W.conj().T)
        key = tuple(np.round(target,6).flatten())
        if key in table:
            return table[key] + seq2
    return None

# 10) Optional beam-search stub
def beam_search(U, gate_dict, beam_width=50, depth=10):
    # Placeholder: user can implement a priority-based expansion
    return None

def euler_decompose(U):
    """
    Given a 2×2 unitary U, return (α, β, γ) so that
    U ≈ Rz(γ) · Rx(β) · Rz(α).
    """
    A = U[0,0]
    B = U[1,1]
    C = U[1,0]
    # 1) β from magnitude of A
    beta = 2*np.arccos(np.clip(np.abs(A), -1, 1))
    # 2) α+γ from phase of U[1,1]
    alpha_plus_gamma = 2*np.angle(B)
    # 3) α−γ from phase of U[1,0] (plus π/2)
    alpha_minus_gamma = 2*(np.angle(C) + np.pi/2)
    # Solve
    alpha = (alpha_plus_gamma + alpha_minus_gamma)/2
    gamma = (alpha_plus_gamma - alpha_minus_gamma)/2
    # Normalize to [0,2π)
    return alpha % (2*np.pi), beta % (2*np.pi), gamma % (2*np.pi)

# helper that invert a gate-sequence
def invert_sequence(seq):
    """
    Given a gate‐name list like ['T','H','Tdg'], return the reversed,
    daggered version: ['T','H','Tdg'] → ['T','H','Tdg']⁻¹ = ['T','H','Tdg']^op
    with H†=H, T†=Tdg, Tdg†=T.
    """
    inv = []
    for g in reversed(seq):
        if   g == 'T':   inv.append('Tdg')
        elif g == 'Tdg': inv.append('T')
        else:            inv.append(g)     # H† = H
    return inv

# helper that calculates gate fidelity
def average_gate_fidelity(U, V):
    d = U.shape[0]
    overlap = np.trace(U.conj().T @ V)
    return (abs(overlap)**2 + d) / (d*(d+1))   # for d=2 gives (|Tr|^2 + 2)/6


In [89]:
# Revised Solovay-Kitaev class
class SolovayKitaev:
    def __init__(self, gate_set, gate_dict, eps0=1e-1, L0=10,
                 refine_K=3, use_mitm=False, use_beam=False, beam_width=50):
        self.G = gate_set
        self.gates = gate_dict
        self.eps0 = eps0
        self.L0 = L0
        self.refine_K = refine_K
        self.use_mitm = use_mitm
        self.use_beam = use_beam
        self.beam_width = beam_width
        self._precomp = precompute_sequences_and_unitaries(self.G, self.L0, self.gates)

    def basic_approximation(self, U):
        # PASS 0: trivial Euler snap (5-gate)
        alpha, beta, gamma = euler_decompose(U)
        k1, k2, k3 = [int(np.round(4*v/np.pi)) for v in (alpha, beta, gamma)]
        seq0 = []
        seq0 += ['T']*k3 if k3>=0 else ['Tdg']*(-k3)
        seq0 += ['H']
        seq0 += ['T']*k2 if k2>=0 else ['Tdg']*(-k2)
        seq0 += ['H']
        seq0 += ['T']*k1 if k1>=0 else ['Tdg']*(-k1)
        V0 = to_su2(compose(seq0, self.gates))
        err0 = distance(U, V0)
        if err0 <= self.eps0:
            return seq0
        bestSeq, bestErr = seq0, err0
        # PASS 1: small-window Euler refine
        for dk1 in range(-self.refine_K, self.refine_K+1):
            for dk2 in range(-self.refine_K, self.refine_K+1):
                for dk3 in range(-self.refine_K, self.refine_K+1):
                    seq = []
                    seq += ['T']*dk3 if dk3>=0 else ['Tdg']*(-dk3)
                    seq += ['H']
                    seq += ['T']*dk2 if dk2>=0 else ['Tdg']*(-dk2)
                    seq += ['H']
                    seq += ['T']*dk1 if dk1>=0 else ['Tdg']*(-dk1)
                    V = to_su2(compose(seq, self.gates))
                    err = distance(U, V)
                    if err < bestErr:
                        bestErr, bestSeq = err, seq
                        if err <= self.eps0:
                            return seq
        # PASS 2: fallback
        if self.use_mitm:
            seq = meet_in_middle(U, self.L0, self.gates)
            if seq is not None:
                return seq
        if self.use_beam:
            seq = beam_search(U, self.gates, self.beam_width, self.L0)
            if seq is not None:
                return seq
        for cand_seq, cand_V in self._precomp:
            err = distance(U, cand_V)
            if err < bestErr:
                bestErr, bestSeq = err, cand_seq
                if err <= self.eps0:
                    return cand_seq
        return bestSeq

    def gc_decompose(self, Delta):
        Delta0 = to_su2(Delta)
        theta, p = AxisAngle(Delta0)
        small_phi = SolveForPhi(theta)
        V0 = Rx(small_phi)
        W0 = Ry(small_phi)
        S = AlignRotation([0,0,1], p)
        V = S @ V0 @ S.conj().T
        W = S @ W0 @ S.conj().T
        return V, W

    def _solve_sk_seq(self, U, n):
        if n == 0:
            return self.basic_approximation(U)
        Un1_seq = self._solve_sk_seq(U, n-1)
        Un1 = compose(Un1_seq, self.gates)
        Delta = U @ Un1.conj().T
        Vmat, Wmat = self.gc_decompose(Delta)
        V_seq = self._solve_sk_seq(Vmat, n-1)
        W_seq = self._solve_sk_seq(Wmat, n-1)
        seq = []
        seq += V_seq
        seq += W_seq
        seq += invert_sequence(V_seq)
        seq += invert_sequence(W_seq)
        seq += Un1_seq
        return seq

    def approximate(self, U, depth):
        seq = self._solve_sk_seq(U, depth)
        V = compose(seq, self.gates)
        F = average_gate_fidelity(U, V)
        return seq, F


In [91]:
U = random_su2()      # your random_unitary(2) returning SU(2)
print(U)

[[-0.55829353-0.24089219j  0.76071927+0.22712439j]
 [-0.76071927+0.22712439j -0.55829353+0.24089219j]]


In [92]:

if __name__=="__main__":
    sk = SolovayKitaev(['H','T'], gate_dict, eps0=0.1, L0=10)
    seq, F = sk.approximate(U, depth=6)
    seq_simpl = simplify_sequence(seq)
    
    # print("Seq:", seq)
    print("Gate depth:", len(seq))
    print("Simplified length:", len(seq_simpl))
    
    print("Fidelity:", F)
    # assert F > 1 - 4*sk.eps0  # rough check

Gate depth: 148095
Simplified length: 91379
Fidelity: 0.4590028777775623


## Multi-qubit SK-algorithm

### Psuedo code for Solovay-Kitaev algorithm for multi-qubit gates


function Solovay-Kitaev(Gate = U, depth = n)

    if n == 0:

        return Basic_Approximation(U)

    else:

        Set $U_{n-1}$ = Solovay-Kitaev(U, n-1)

        Set $V, W$ = GC-Approx-Decompose($U U^\dagger_{n-1}$)

        Set $V_{n-1}$ = Solovay-Kitaev(V, n-1)

        Set $W_{n-1}$ = Solovay-Kitaev(W, n-1)

        return $U_n = V_{n-1} W_{n-1} V^\dagger_{n-1} W^\dagger_{n-1} U_{n-1}$

